In [1]:
import pandas as pd
cereales=pd.read_csv("FAOSTATcereales.csv")
MoyenneEnergiePoids=pd.read_csv("MoyenneEnergiePoids.csv",index_col=0)
RatioProteinePoids=pd.read_csv("RatioProteinePoids.csv",index_col=0)

In [2]:
ListeProduitsCereales=cereales.drop(columns=["Code Domaine","Domaine","Code Pays","Pays","Code Élément","Élément","Code Année","Année","Unité","Valeur","Symbole","Description du Symbole"])

In [3]:
ListeProduitsCereales=ListeProduitsCereales.drop_duplicates()
#Liste des produits de type Céréales et leur codes
ListeProduitsCereales

,Code Produit,Produit
0,2511,Blé
1,2805,Riz (Eq Blanchi)
2,2513,Orge
4,2514,Maïs
6,2517,Millet
7,2520,"Céréales, Autres"
15,2515,Seigle
17,2516,Avoine
21,2518,Sorgho


In [4]:
CerealesPourAnimaux=cereales.groupby("Élément").get_group("Aliments pour animaux")

In [5]:
CerealesPourAnimauxClean=CerealesPourAnimaux.drop(columns=["Code Domaine","Code Produit","Domaine","Code Pays","Pays","Code Élément","Élément","Code Année","Année","Unité","Symbole","Description du Symbole"])

In [6]:
CerealesPourAnimauxClean["Valeur"]=CerealesPourAnimauxClean["Valeur"]*1000000
#Conversion des milliers de tonnes en kilos
CerealesPourAnimauxClean=CerealesPourAnimauxClean.rename(columns={"Valeur":"Kilos"})

In [7]:
CerealesPourAnimauxClean=CerealesPourAnimauxClean.groupby("Produit").sum().reset_index()
#Pour obtenir la somme pour chaque produit de toute la disponibilité des céréales 
#pour l'alimentation des animaux en kilos par an

In [8]:
CerealesPourAnimauxClean
#Céréales qui partent pour l'alimentation animale

,Produit,Kilos
0,Avoine,16251000000
1,Blé,129668000000
2,"Céréales, Autres",19035000000
3,Maïs,546116000000
4,Millet,3306000000
5,Orge,92658000000
6,Riz (Eq Blanchi),33594000000
7,Seigle,8099000000
8,Sorgho,24808000000


In [9]:
#Pour la conversion en Kcal j'importe puis fusionne avec la table MoyenneEnergiePoids, calculée précédemment
CerealesPourAnimEnKcal=pd.merge(CerealesPourAnimauxClean,MoyenneEnergiePoids,on="Produit")

In [10]:
#Je crée la colonne et effectue le calcul
CerealesPourAnimEnKcal["Kcal"]=CerealesPourAnimEnKcal.Kilos*CerealesPourAnimEnKcal.RatioEnergiePoids

In [11]:
CerealesPourAnimEnKcal
#Données converties en Kcal

,Produit,Kilos,RatioEnergiePoids,Kcal
0,Avoine,16251000000,2273.494107,3.694655e+13
1,Blé,129668000000,2849.720839,3.695176e+14
2,"Céréales, Autres",19035000000,2912.034307,5.543057e+13
3,Maïs,546116000000,3021.790252,1.650248e+15
4,Millet,3306000000,3103.181312,1.025912e+13
5,Orge,92658000000,2551.209457,2.363900e+14
6,Riz (Eq Blanchi),33594000000,3640.903953,1.223125e+14
7,Seigle,8099000000,2745.460587,2.223549e+13
8,Sorgho,24808000000,3106.869396,7.707522e+13


In [12]:
#Pour la conversion en protéines j'importe puis fusionne avec la table RatioProteinePoids, calculée précédemment
ProtCerealesPourAnim=pd.merge(RatioProteinePoids,CerealesPourAnimauxClean,on="Produit")

In [13]:
#Je crée la colonne et effectue le calcul
ProtCerealesPourAnim["Protéines en grammes"]=(ProtCerealesPourAnim.Pourcentage/100)*ProtCerealesPourAnim.Kilos*1000

In [14]:
ProtCerealesPourAnim
#Données converties en protéines

,Produit,Pourcentage,RatioProteinePoids,Kilos,Protéines en grammes
0,Avoine,8.301523,8.301523,16251000000,1.349081e+12
1,Blé,8.855398,8.855398,129668000000,1.148262e+13
2,"Céréales, Autres",8.477314,8.477314,19035000000,1.613657e+12
3,Maïs,7.310310,7.310310,546116000000,3.992277e+13
4,Millet,7.898374,7.898374,3306000000,2.611202e+11
5,Orge,7.231703,7.231703,92658000000,6.700751e+12
6,Riz (Eq Blanchi),6.860267,6.860267,33594000000,2.304638e+12
7,Seigle,7.344320,7.344320,8099000000,5.948165e+11
8,Sorgho,8.904038,8.904038,24808000000,2.208914e+12


In [15]:
#Pour la dernière question, je reprends le fichier du début dont j'isole l'élément "Disponibilité intérieure"
CerealesDispoInt=cereales.groupby("Élément").get_group("Disponibilité intérieure")
CerealesDispoInt=CerealesDispoInt.drop(columns=["Code Domaine","Code Produit","Domaine","Code Pays","Code Élément","Élément","Code Année","Année","Unité","Symbole","Description du Symbole"])
CerealesDispoInt["Valeur"]=CerealesDispoInt["Valeur"]*1000000
#Conversion des milliers de tonnes en kilos
CerealesDispoInt=CerealesDispoInt.rename(columns={"Valeur":"Kilos"})

In [16]:
CerealesDispoInt.sum()["Kilos"]
#Totale de la disponibilité intérieure des céréales en kilos
#2,4 milliards de tonnes par an

2406999000000

In [17]:
CerealesPourAnimauxClean.sum()["Kilos"]
#Totale de la disponibilité des céréales pour l'alimentation animale en kilos
#873 millions de tonnes par an

873535000000

In [18]:
CerealesPourAnimauxClean.sum()["Kilos"]/CerealesDispoInt.sum()["Kilos"]*100
#Pourcentage de la disponibilité mondiale de céréales déstinée à l'alimentation animale

36.291456706047654